<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>


# Lab 3.2.2 
# *Mining Social Media with Twitter*

## The Twitter API and Tweepy Package

The Twitter API provides access to tweets and comments, and allows an application to post tweets to the user's timeline. 

Twitter requires developers to create and authenticate an app before they can use the API. 

### 1. Apply for Developer Access

Go to https://blog.twitter.com/developer/en_us/topics/tools/2018/new-developer-requirements-to-protect-our-platform.html
and read the advice.


Apply at https://developer.twitter.com/en/apply-for-access.html. Where asked state that you will use the app as a student to explore the tweepy Python library and use it to fetch recent tweets and to create/delete a tweet.

### 2. Create Your Twitter App
Go to https://developer.twitter.com/en/portal/projects-and-apps and click on the "+ New Project" button. Give your project a name, select Student as the use case and enter a brief project description. Then click on the "Create new" button to create a new App. Choose a "Development" app environment, give your app a name and you will be able to see your API Key, API Key Secret and Bearer Token. You will also be able to generate an Access Token and Secret. Copy-paste these authentication details for use in this lab.

### 3. Load Python Libraries

In [ ]:
!pip install tweepy
!pip install textblob

In [ ]:
import tweepy
import json
import pprint
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob

Ensure your version of tweepy is 4.5.0 or later:

In [ ]:
tweepy.__version__

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [ ]:
bearer_token="" #your bearer token (string) goes in here
api_consumer_key = "" # your consumer key (string) goes in here
api_consumer_secret = "" # your consumer secret key (string) goes in here
access_token = ""  # your access token (string) goes in here
access_token_secret = ""  # your access token secret (string) goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_twitter.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{`<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`   "bearer_token": "your bearer token (string) goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`   "consumer_key": "your consumer key (string) goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "consumer_secret": "your consumer secret key (string) goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"access_token": "your access token (string) goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"access_token_secret": "your access token secret (string) goes in here"` <br>
`}`

(Nb. Parsers are very fussy. Make sure each key:value pair has a comma after it except the last one!)  

Use the following code to load the credentials:  

In [ ]:
pwd()  # make sure your working directory is where the file is

In [ ]:
path_auth = 'auth_twitter.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:
#pp.pprint(auth)

my_bearer_token = auth['bearer_token']
my_consumer_key = auth['consumer_key']
my_consumer_secret = auth['consumer_secret']
my_access_token = auth['access_token']
my_access_token_secret = auth['access_token_secret']

Security considerations: 
- this method only keeps your credentials invisible as long as nobody accesses this notebook while it's running on your computer 
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Twitter using the Tweepy library:

In [ ]:
client = tweepy.Client(
    bearer_token=my_bearer_token,
    consumer_key=my_consumer_key,
    consumer_secret=my_consumer_secret,
    access_token=my_access_token,
    access_token_secret=my_access_token_secret
)

In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [ ]:
client.

To obtain your followers, enter your Twitter id. An easy way to find this is via https://tweeterid.com/.

In [ ]:
myid = '' #enter your id
followers = client.get_users_followers(id=myid)

In [ ]:
followers.data[0].#try id, name, username

You can enter a query such as the following to look up recent tweets based on a search string (excluding retweets).

In [ ]:
query = 'winterolympics -is:retweet'

tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'],
                                     media_fields=['preview_image_url'], expansions='attachments.media_keys',
                                     max_results=100)

In [ ]:
tweets.data[0].

Consult the Tweepy and Twitter API documentation. Print a few of the response members below:

In [ ]:
username = '' #choose a username
query = 'from:' + username
recent_tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['attachments','author_id','context_annotations','created_at','entities','geo','id','in_reply_to_user_id','lang','possibly_sensitive','public_metrics','referenced_tweets','source','text','withheld'],
                                     max_results=10)

The following few cells will fetch recent tweets from accounts you follow:

In [ ]:
users = client.get_users_following(id=myid, max_results = 1000, user_fields=['profile_image_url'])
print(len(users.data)) #number of accounts you follow

In [ ]:
following_ids = []
for user in users.data:
    following_ids.append(user.id)

If you do not follow any accounts, create a list called following_ids with a list of ids of interest that can be looked up from https://tweeterid.com/.

In [ ]:
for f in following_ids[:10]: #the number 10 can be increased to show tweets from more accounts
    query = 'from:' + str(f) + ' -is:retweet'

# get max. 10 tweets from each account followed
    tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=10)
    if tweets.data is not None:
        for tweet in tweets.data:
            print(tweet.author_id, tweet.created_at, tweet.text)

The request to see your own recent tweets is similar, but uses a single call to the `get_users_tweets` endpoint. Try this below:

Now, instead of printing the text of each tweet, print the `created_at` and `id` attributes:

You can create a tweet as follows:

In [ ]:
# create a tweet:
tweet = client.create_tweet(text='Test: Made with Tweepy')

(Nb. Don't abuse this feature! If you try to generate a zillion tweets in a loop, Twitter will ban your account.)

Tweets can be deleted by reference to their `id_str` attribute:

In [ ]:
tweet.data['id']

In [ ]:
# delete a tweet:
status = client.delete_tweet(tweet.data['id'])

You can follow a Tweeter:

In [ ]:
# follow:
client.follow_user('10228272') #YouTube

or unfollow:

In [ ]:
# unfollow:
client.unfollow_user('10228272')



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



